In [1]:
import shutil

import duckdb
import lance
import numpy as np
import pandas as pd
import pyarrow as pa

## Creating datasets

Via pyarrow it's really easy to create lance datasets

Create a dataframe

In [2]:
df = pd.DataFrame({"a": [5]})
df

,a
0,5


Write it to lance

In [3]:
shutil.rmtree("/tmp/test_df.lance", ignore_errors=True)

dataset = lance.write_dataset(df, "/tmp/test_df.lance")
dataset.to_table().to_pandas()

,a
0,5


### Converting from parquet

In [4]:
shutil.rmtree("/tmp/test.parquet", ignore_errors=True)
shutil.rmtree("/tmp/test_parquet.lance", ignore_errors=True)

tbl = pa.Table.from_pandas(df)
pa.dataset.write_dataset(tbl, "/tmp/test.parquet", format='parquet')

parquet = pa.dataset.dataset("/tmp/test.parquet")
parquet.to_table().to_pandas()

,a
0,5


Write to lance in 1 line

In [5]:
dataset = lance.write_dataset(parquet, "/tmp/test_parquet.lance")

In [6]:
# make sure it's the same
dataset.to_table().to_pandas()

,a
0,5


## Versioning

We can append rows

In [7]:
df = pd.DataFrame({"a": [10]})
tbl = pa.Table.from_pandas(df)
dataset = lance.write_dataset(tbl, "/tmp/test.lance", mode="append")

dataset.to_table().to_pandas()

[2024-07-31T13:51:06Z WARN  lance::dataset] No existing dataset at /tmp/test.lance, it will be created


,a
0,10


We can overwrite the data and create a new version

In [8]:
df = pd.DataFrame({"a": [50, 100]})
tbl = pa.Table.from_pandas(df)
dataset = lance.write_dataset(tbl, "/tmp/test.lance", mode="overwrite")

In [9]:
dataset.to_table().to_pandas()

,a
0,50
1,100


The old version is still there

In [10]:
dataset.versions()

[{'version': 1,
  'timestamp': datetime.datetime(2024, 7, 31, 23, 21, 6, 355179),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2024, 7, 31, 23, 21, 6, 374206),
  'metadata': {}}]

In [11]:
lance.dataset('/tmp/test.lance', version=1).to_table().to_pandas()

,a
0,10


In [12]:
lance.dataset('/tmp/test.lance', version=2).to_table().to_pandas()

,a
0,50
1,100


We can create tags

In [13]:
dataset.tags.create("stable", 1)
dataset.tags.create("nightly", 2)
dataset.tags.list()

{'stable': {'version': 1, 'manifest_size': 613},
 'nightly': {'version': 2, 'manifest_size': 613}}

which can be checked out

In [14]:
lance.dataset('/tmp/test.lance', version="stable").to_table().to_pandas()

,a
0,10


## Vectors

### Data preparation

For this tutorial let's use the Sift 1M dataset:

- Download `ANN_SIFT1M` from: http://corpus-texmex.irisa.fr/
- Direct link should be `ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz`
- Download and then unzip the tarball

In [15]:
!rm -rf sift* vec_data.lance
!wget ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz
!tar -xzf sift.tar.gz

--2024-07-31 23:21:07--  ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz
           => ‘sift.tar.gz’
Resolving ftp.irisa.fr (ftp.irisa.fr)... 131.254.254.45, 2001:660:7303:254::45
Connecting to ftp.irisa.fr (ftp.irisa.fr)|131.254.254.45|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /local/texmex/corpus ... done.
==> SIZE sift.tar.gz ... 168280445
==> PASV ... done.    ==> RETR sift.tar.gz ... done.
Length: 168280445 (160M) (unauthoritative)

sift.tar.gz         100%[===================>] 160.48M  5.71MB/s    in 38s     

2024-07-31 23:21:49 (4.20 MB/s) - ‘sift.tar.gz’ saved [168280445]



Convert it to Lance

In [16]:
from lance.vector import vec_to_table
import struct

uri = "vec_data.lance"

with open("sift/sift_base.fvecs", mode="rb") as fobj:
    buf = fobj.read()
    data = np.array(struct.unpack("<128000000f", buf[4 : 4 + 4 * 1000000 * 128])).reshape((1000000, 128))
    dd = dict(zip(range(1000000), data))

table = vec_to_table(dd)
lance.write_dataset(table, uri, max_rows_per_group=8192, max_rows_per_file=1024*1024)

In [17]:
uri = "vec_data.lance"
sift1m = lance.dataset(uri)

### KNN (no index)

Sample 100 vectors as query vectors

In [18]:
samples = duckdb.query("SELECT vector FROM sift1m USING SAMPLE 100").to_df().vector
samples

0     [6.0, 0.0, 0.0, 0.0, 0.0, 28.0, 131.0, 53.0, 0...
1     [0.0, 0.0, 0.0, 20.0, 37.0, 9.0, 0.0, 0.0, 0.0...
2     [0.0, 0.0, 0.0, 0.0, 19.0, 86.0, 121.0, 40.0, ...
3     [2.0, 0.0, 0.0, 20.0, 123.0, 43.0, 0.0, 0.0, 0...
4     [110.0, 19.0, 7.0, 17.0, 1.0, 0.0, 10.0, 48.0,...
                            ...                        
95    [7.0, 6.0, 8.0, 0.0, 8.0, 23.0, 1.8e-43, 42.0,...
96    [1.0, 0.0, 0.0, 2.0, 7.0, 0.0, 17.0, 23.0, 43....
97    [64.0, 126.0, 127.0, 1.0, 0.0, 0.0, 4.0, 127.0...
98    [42.0, 2.0, 3.0, 8.0, 1.8e-43, 6.0, 0.0, 0.0, ...
99    [0.0, 7.0, 123.0, 36.0, 27.0, 4.0, 0.0, 5.0, 3...
Name: vector, Length: 100, dtype: object

Call nearest neighbors (no ANN index here)

In [19]:
import time

start = time.time()
tbl = sift1m.to_table(columns=["id"], nearest={"column": "vector", "q": samples[0], "k": 10})
end = time.time()

print(f"Time(sec): {end-start}")
print(tbl.to_pandas())

Time(sec): 0.5790560245513916
       id  _distance
0  376447        0.0
1  662827    19363.0
2  672502    23895.0
3  899800    24001.0
4  913474    24856.0
5  612001    26252.0
6  827689    26438.0
7  678694    26792.0
8  660247    29417.0
9  869872    30206.0


Without the index this is scanning through the whole dataset to compute the distance. <br/>

For real-time serving we can do much better with an ANN index

### Build index

Now let's build an index. Lance now supports IVF_PQ, IVF_HNSW_PQ and IVF_HNSW_SQ indexes

**NOTE** If you'd rather not wait for index build, you can download a version with the index pre-built from [here](https://eto-public.s3.us-west-2.amazonaws.com/datasets/sift/sift_ivf256_pq16.tar.gz) and skip the next cell

In [20]:
%%time 

sift1m.create_index(
    "vector",
    index_type="IVF_PQ", # IVF_PQ, IVF_HNSW_PQ and IVF_HNSW_SQ are supported
    num_partitions=256,  # IVF
    num_sub_vectors=16,  # PQ
)

**NOTE** If you're trying this on your own data, make sure your vector (dimensions / num_sub_vectors) % 8 == 0, or else index creation will take much longer than expected due to SIMD misalignment

### Try nearest neighbors again with ANN index

Let's look for nearest neighbors again

In [21]:
sift1m = lance.dataset(uri)

In [22]:
import time

tot = 0
for q in samples:
    start = time.time()
    tbl = sift1m.to_table(nearest={"column": "vector", "q": q, "k": 10})
    end = time.time()
    tot += (end - start)

print(f"Avg(sec): {tot / len(samples)}")
print(tbl.to_pandas())

Avg(sec): 0.7772157430648804
       id                                             vector  _distance
0  117815  [0.0, 7.0, 123.0, 36.0, 27.0, 4.0, 0.0, 5.0, 3...        0.0
1  453215  [11.0, 8.0, 14.0, 23.0, 54.0, 1.0, 0.0, 0.0, 3...    98424.0
2  146711  [11.0, 36.0, 98.0, 110.0, 15.0, 6.0, 0.0, 0.0,...   112725.0
3  732500  [0.0, 4.0, 88.0, 67.0, 37.0, 63.0, 0.0, 0.0, 0...   113449.0
4  352724  [47.0, 74.0, 55.0, 14.0, 21.0, 34.0, 0.0, 0.0,...   117533.0
5  296480  [17.0, 39.0, 82.0, 33.0, 0.0, 81.0, 4.0, 0.0, ...   118344.0
6  873368  [22.0, 33.0, 3.0, 0.0, 84.0, 16.0, 0.0, 0.0, 4...   119968.0
7  304478  [0.0, 20.0, 108.0, 80.0, 42.0, 2.0, 4.0, 4.0, ...   120192.0
8  554738  [10.0, 22.0, 22.0, 43.0, 63.0, 11.0, 0.0, 25.0...   121118.0
9  154064  [0.0, 22.0, 120.0, 70.0, 22.0, 33.0, 2.0, 0.0,...   123900.0


**NOTE** on performance, your actual numbers will vary by your storage. These numbers are run on local disk on an M1 Macbook Air. If you're querying S3 directly, HDD, or network drives, performance will be slower.

The latency vs recall is tunable via:
- nprobes: how many IVF partitions to search
- refine_factor: determines how many vectors are retrieved during re-ranking

In [23]:
%%time

sift1m.to_table(
    nearest={
        "column": "vector",
        "q": samples[0],
        "k": 10,
        "nprobes": 10,
        "refine_factor": 5,
    }
).to_pandas()

CPU times: user 2.79 s, sys: 271 ms, total: 3.07 s
Wall time: 729 ms


,id,vector,_distance
0,376447,"[6.0, 0.0, 0.0, 0.0, 0.0, 28.0, 131.0, 53.0, 0...",0.0
1,662827,"[11.0, 0.0, 0.0, 1.0, 26.0, 39.0, 120.0, 42.0,...",19363.0
2,672502,"[4.0, 0.0, 0.0, 0.0, 0.0, 15.0, 138.0, 52.0, 0...",23895.0
3,899800,"[11.0, 0.0, 0.0, 0.0, 0.0, 8.0, 133.0, 79.0, 0...",24001.0
4,913474,"[8.0, 0.0, 0.0, 0.0, 0.0, 43.0, 136.0, 18.0, 0...",24856.0
5,612001,"[4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 127.0, 48.0, 0....",26252.0
6,827689,"[8.0, 0.0, 0.0, 0.0, 0.0, 49.0, 116.0, 49.0, 0...",26438.0
7,678694,"[22.0, 5.0, 6.0, 0.0, 0.0, 41.0, 133.0, 84.0, ...",26792.0
8,660247,"[29.0, 0.0, 0.0, 4.0, 30.0, 51.0, 96.0, 64.0, ...",29417.0
9,869872,"[1.0, 0.0, 0.0, 0.0, 0.0, 17.0, 126.0, 64.0, 0...",30206.0


q => sample vector

k => how many neighbors to return

nprobes => how many partitions (in the coarse quantizer) to probe

refine_factor => controls "re-ranking". If k=10 and refine_factor=5 then retrieve 50 nearest neighbors by ANN and re-sort using actual distances then return top 10. This improves recall without sacrificing performance too much

**NOTE** the latencies above include file io as lance currently doesn't hold anything in memory. Along with index building speed, creating a purely in memory version of the dataset would make the biggest impact on performance.

### Features and vector can be retrieved together

Usually we have other feature or metadata columns that need to be stored and fetched together.
If you're managing data and the index separately, you have to do a bunch of annoying plumbing to put stuff together. With Lance it's a single call

In [24]:
tbl = sift1m.to_table()
tbl = tbl.append_column("item_id", pa.array(range(len(tbl))))
tbl = tbl.append_column("revenue", pa.array((np.random.randn(len(tbl))+5)*1000))
tbl.to_pandas()

,id,vector,item_id,revenue
0,0,"[0.0, 16.0, 35.0, 5.0, 32.0, 31.0, 14.0, 10.0,...",0,4805.543755
1,1,"[1.8e-43, 14.0, 35.0, 19.0, 20.0, 3.0, 1.0, 13...",1,6169.830775
2,2,"[33.0, 1.8e-43, 0.0, 1.0, 5.0, 3.0, 44.0, 40.0...",2,3249.604494
3,3,"[23.0, 10.0, 1.8e-43, 12.0, 47.0, 14.0, 25.0, ...",3,5651.837169
4,4,"[27.0, 29.0, 21.0, 1.8e-43, 1.0, 1.0, 0.0, 0.0...",4,5338.017959
...,...,...,...,...
999995,999995,"[8.0, 9.0, 5.0, 0.0, 10.0, 39.0, 72.0, 68.0, 3...",999995,4823.274436
999996,999996,"[3.0, 28.0, 55.0, 29.0, 35.0, 12.0, 1.0, 2.0, ...",999996,3223.938607
999997,999997,"[0.0, 13.0, 41.0, 72.0, 40.0, 9.0, 0.0, 0.0, 0...",999997,5533.752357
999998,999998,"[41.0, 121.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24...",999998,3923.410822


In [25]:
sift1m = lance.write_dataset(tbl, uri, mode="overwrite")

In [26]:
sift1m.to_table(columns=["revenue"], nearest={"column": "vector", "q": samples[0], "k": 10}).to_pandas()

,revenue,_distance
0,4827.212670,0.0
1,3242.904851,19363.0
2,5269.286209,23895.0
3,3812.754778,24001.0
4,5008.788640,24856.0
5,4801.355328,26252.0
6,5352.014501,26438.0
7,4746.360670,26792.0
8,3784.408760,29417.0
9,4158.796077,30206.0
